# CausalTorch: Text Generation with Causal Constraints

This notebook demonstrates how to use the CausalTorch library to generate text with causal constraints. The CNSG-Net model combines a neural language model with symbolic causal rules to ensure logical consistency in the generated text.

In [ ]:
import os
import torch
import json
from causaltorch import CNSG_Net
from causaltorch.rules import CausalRuleEngine
from causaltorch.utils import calculate_rule_violation_rate

# Check for CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 1. Create Causal Rules

First, we'll define some physical causal rules for our text generation model.

In [ ]:
# Create a directory for our rules if it doesn't exist
os.makedirs("rules", exist_ok=True)

# Define some simple physical causal rules
physical_rules = [
    {
        "name": "rain_wet_ground",
        "description": "If it rains, the ground gets wet",
        "pattern": "rain|raining|rainy|downpour",
        "consequences": [
            {
                "text": "wet",
                "intensity": 5.0,
                "required": True
            },
            {
                "text": "puddle",
                "intensity": 3.0,
                "required": False
            }
        ],
        "type": "physical"
    },
    {
        "name": "fire_heat",
        "description": "Fire produces heat",
        "pattern": "fire|flame|burning",
        "consequences": [
            {
                "text": "heat",
                "intensity": 4.0,
                "required": True
            },
            {
                "text": "warm",
                "intensity": 3.0,
                "required": False
            }
        ],
        "type": "physical"
    }
]

# Save rules to file
rules_file = "rules/physical_rules.json"
with open(rules_file, "w") as f:
    json.dump(physical_rules, f, indent=2)

print(f"Saved {len(physical_rules)} rules to {rules_file}")

## 2. Initialize the CNSG-Net Model

Now we'll initialize the Causal Neural-Symbolic Generative Network with the rules we defined.

In [ ]:
# Initialize the model with our physical rules
model = CNSG_Net(
    base_model_name="gpt2",  # We'll use GPT-2 small as the base model
    rules_file=rules_file
)

# Move to GPU if available
model = model.to(device)

print("Model initialized successfully!")

## 3. Generate Text with Causal Constraints

Let's test our model by generating text that should follow our defined causal rules.

In [ ]:
prompts = [
    "It started raining heavily. The ground",
    "The fire was burning brightly in the fireplace. I could feel the",
    "Despite the heavy rain, the ground remained",  # This should be interesting - will it respect causality?
    "The match struck the surface and a fire started. The room began to feel"
]

generated_texts = []

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    output = model.generate(
        prompt, 
        max_length=100,
        do_sample=True,
        temperature=0.7
    )
    print(f"Generated: {output}")
    generated_texts.append(output)

## 4. Evaluate Causal Consistency

Now let's evaluate how well our model respects the causal constraints.

In [ ]:
# Calculate violation rate
violation_stats = calculate_rule_violation_rate(
    model.rule_engine,
    generated_texts,
    prompts
)

print(f"Violation rate: {violation_stats['violation_rate'] * 100:.2f}%")
if violation_stats['violations_by_rule']:
    print("Violations by rule:")
    for rule, count in violation_stats['violations_by_rule'].items():
        print(f"  - {rule}: {count}")
else:
    print("No violations detected! All causal rules were respected.")

## 5. Compare with Baseline (No Causal Rules)

For comparison, let's see how a standard language model (without causal constraints) generates text for the same prompts.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load standard GPT-2 model without causal constraints
baseline_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
baseline_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

baseline_generated = []

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    input_ids = baseline_tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Generate text
    output_ids = baseline_model.generate(
        input_ids,
        max_length=100,
        do_sample=True,
        temperature=0.7
    )
    
    output = baseline_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"Baseline Generated: {output}")
    baseline_generated.append(output)

# Calculate violation rate for baseline model
baseline_violation_stats = calculate_rule_violation_rate(
    model.rule_engine,  # Use the same rule engine to evaluate
    baseline_generated,
    prompts
)

print(f"\nBaseline violation rate: {baseline_violation_stats['violation_rate'] * 100:.2f}%")
if baseline_violation_stats['violations_by_rule']:
    print("Violations by rule:")
    for rule, count in baseline_violation_stats['violations_by_rule'].items():
        print(f"  - {rule}: {count}")

## 6. Conclusion

We've seen how the CNSG-Net model can generate text with improved causal consistency compared to a standard language model. By encoding causal rules, we ensure that generated text follows logical patterns like "rain causes wet ground" and "fire produces heat".

This approach allows for more reliable and trustworthy text generation, especially for domains where logical consistency is critical.